In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


 This particular solution is lsightly tricky because here we need to decide what we want to achieve here we need to balance out the situation of identfying the junk property properly 
 so there would be certain cases with the junk property writeen as non junk property and there would be certain cases where the good propert might be classified as  as junk property 
 
 so we need to make sure based on our understanding and our business requirements if we want to classify all the junk property or we still need to keep all the non junk properrty 
 
 so let's say if it is important for us to make sure that no junk property is classified as junk then then tehere will be some case where the property are not junk and still be classified as junk property 
 
 becuase we will kepp here some thresold such that some non junk property would be classified as junk , but it would make us sure that there no junk property is skipped or left 
 
 
 Another scenario could be , let's say it's fine if we have a few junk properties listed but we don't want to give up on good property so in that case we will keep our thresold such that the good properties are all taagged correctly and and some bad or junk property will come a good property 

In [44]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings("ignore")
import pandas_profiling as pp
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix, fbeta_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report as cr

import matplotlib.pyplot as plt
plt.style.use("ggplot")
df = pd.read_csv('/content/drive/My Drive/collab projects/Classification Project Industry Dataset/Property_train.csv', parse_dates=True)
df.head()

,Junk,InteriorsStyle,PriceIndex8,ListDate,Material,PriceIndex9,Agency,AreaIncomeType,EnvRating,PriceIndex7,ExpeditedListing,PriceIndex4,PriceIndex1,PriceIndex6,PRIMEUNIT,Channel,Zip,InsurancePremiumIndex,PlotType,Architecture,PriceIndex3,Region,PriceIndex5,SubModel,Facade,State,NormalisedPopulation,BuildYear,RegionType,PropertyAge,PriceIndex2
0,0,6LLJ,14674.0,9/7/2010,UT7W,8270,CAT3,B,missing,13143.0,0,14224.0,9217.0,10387.0,missing,Direct,21075,623,WHBI,I3Z9,13108.0,A,9022.0,6E9G,GREEN,MD,42077,2008,A,2,10692.0
1,1,XZ9F,4172.0,1/7/2009,PRN0,3890,CAT2,B,missing,3461.0,0,4404.0,2958.0,3400.0,missing,Direct,73129,1689,WHBI,AVYP,3695.0,A,2742.0,FIVQ,SILVER,OK,77258,2002,A,7,3615.0
2,0,QWEV,8210.0,6/23/2010,A9RA,5900,CAT1,B,missing,6922.0,0,8952.0,4568.0,4862.0,missing,Other,29697,2351,W62B,1M5X,7730.0,A,3768.0,4K0H,SILVER,SC,94514,2004,A,6,5805.0
3,0,MK2K,7309.0,3/19/2009,RRJQ,7460,OTHER,A,missing,6300.0,0,7460.0,5361.0,6305.0,missing,Agent,28273,1933,WLQ6,DKOO,6290.0,A,5370.0,ZLXY,WHITE,NC,82302,2002,C,7,6444.0
4,0,GK4G,9182.0,3/3/2010,HE5A,6400,OTHER,B,missing,8330.0,0,9846.0,5428.0,5718.0,missing,Other,29697,482,WHBI,KML6,8583.0,A,4803.0,FIVQ,SILVER,SC,49176,2007,B,3,6286.0


So in the data we have flag of junks 0 and 1 

Then there are different columns which are inetrior style , priceindex, listdate 

so we are not domain expert sow e don't know much about the column so we will have to analyaze the column and then find out what is important and what is not 

In [45]:
print(df.shape)

(62035, 31)


In [46]:
df.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'Zip', 'InsurancePremiumIndex', 'PlotType',
       'Architecture', 'PriceIndex3', 'Region', 'PriceIndex5', 'SubModel',
       'Facade', 'State', 'NormalisedPopulation', 'BuildYear', 'RegionType',
       'PropertyAge', 'PriceIndex2'],
      dtype='object')

In [47]:
df.describe()

,Junk,PriceIndex9,ExpeditedListing,Zip,InsurancePremiumIndex,NormalisedPopulation,BuildYear,PropertyAge
count,62035.000000,62035.000000,62035.000000,62035.000000,62035.000000,62035.000000,62035.000000,62035.00000
mean,0.122544,6731.444765,0.024680,58038.149109,1277.286854,71462.694302,2005.347739,4.17171
std,0.327915,1766.130806,0.155148,26143.635828,600.787708,14583.079578,1.730014,1.70878
min,0.000000,225.000000,0.000000,2764.000000,462.000000,4825.000000,2001.000000,0.00000
25%,0.000000,5430.000000,0.000000,32124.000000,837.000000,61803.500000,2004.000000,3.00000
50%,0.000000,6705.000000,0.000000,73108.000000,1155.000000,73361.000000,2005.000000,4.00000
75%,0.000000,7900.000000,0.000000,80022.000000,1623.000000,82370.500000,2007.000000,5.00000
max,1.000000,38785.000000,1.000000,99224.000000,7498.000000,115717.000000,2010.000000,9.00000


So from above we can easily identify the outliers easily 

so in priceindex9 the 75% is 7900 and max value is very high in comparison to 75% value which surly shows that there are few outliers present 

so we can check other columns also 

In [48]:
list(zip(df.columns,df.dtypes,df.nunique()))

[('Junk', dtype('int64'), 2),
 ('InteriorsStyle', dtype('O'), 1036),
 ('PriceIndex8', dtype('O'), 12686),
 ('ListDate', dtype('O'), 516),
 ('Material', dtype('O'), 134),
 ('PriceIndex9', dtype('int64'), 2019),
 ('Agency', dtype('O'), 5),
 ('AreaIncomeType', dtype('O'), 4),
 ('EnvRating', dtype('O'), 3),
 ('PriceIndex7', dtype('O'), 12078),
 ('ExpeditedListing', dtype('int64'), 2),
 ('PriceIndex4', dtype('O'), 13006),
 ('PriceIndex1', dtype('O'), 10053),
 ('PriceIndex6', dtype('O'), 10863),
 ('PRIMEUNIT', dtype('O'), 3),
 ('Channel', dtype('O'), 3),
 ('Zip', dtype('int64'), 152),
 ('InsurancePremiumIndex', dtype('int64'), 277),
 ('PlotType', dtype('O'), 13),
 ('Architecture', dtype('O'), 33),
 ('PriceIndex3', dtype('O'), 12309),
 ('Region', dtype('O'), 3),
 ('PriceIndex5', dtype('O'), 10011),
 ('SubModel', dtype('O'), 831),
 ('Facade', dtype('O'), 17),
 ('State', dtype('O'), 37),
 ('NormalisedPopulation', dtype('int64'), 36675),
 ('BuildYear', dtype('int64'), 10),
 ('RegionType', dtype(

In [49]:
cutoff = (5/100)* df.shape[0]
cutoff

3101.75

so here we have decided cut of value as 5%

so anything which ahs value or categories less than 3101.75 we will ingnore that particular category 

In [50]:
numcol = ["PriceIndex5","PriceIndex7","PriceIndex4","PriceIndex1","PriceIndex6","PriceIndex2","PriceIndex3","PriceIndex8", "PriceIndex9"]
for col in numcol:
  df[col] = pd.to_numeric(df[col], errors="coerce")
list(zip(df.columns,df.dtypes,df.nunique()))

[('Junk', dtype('int64'), 2),
 ('InteriorsStyle', dtype('O'), 1036),
 ('PriceIndex8', dtype('float64'), 12685),
 ('ListDate', dtype('O'), 516),
 ('Material', dtype('O'), 134),
 ('PriceIndex9', dtype('int64'), 2019),
 ('Agency', dtype('O'), 5),
 ('AreaIncomeType', dtype('O'), 4),
 ('EnvRating', dtype('O'), 3),
 ('PriceIndex7', dtype('float64'), 12077),
 ('ExpeditedListing', dtype('int64'), 2),
 ('PriceIndex4', dtype('float64'), 13005),
 ('PriceIndex1', dtype('float64'), 10052),
 ('PriceIndex6', dtype('float64'), 10862),
 ('PRIMEUNIT', dtype('O'), 3),
 ('Channel', dtype('O'), 3),
 ('Zip', dtype('int64'), 152),
 ('InsurancePremiumIndex', dtype('int64'), 277),
 ('PlotType', dtype('O'), 13),
 ('Architecture', dtype('O'), 33),
 ('PriceIndex3', dtype('float64'), 12308),
 ('Region', dtype('O'), 3),
 ('PriceIndex5', dtype('float64'), 10010),
 ('SubModel', dtype('O'), 831),
 ('Facade', dtype('O'), 17),
 ('State', dtype('O'), 37),
 ('NormalisedPopulation', dtype('int64'), 36675),
 ('BuildYear', d

There are certain numerical columns which are in catagorical form so we will vonvert that into numerical form 

In [27]:
cols = df.select_dtypes(["object"])
for colname in cols:
  cats = df[colname].value_counts()
  keepcat = cats.index[cats >= cutoff]
  for cat in keepcat:
    if cat == "missing":
      continue
    name=colname+'_'+cat
    df[name] = (df[colname]==cat).astype(int)
  del df[colname]
  print(colname)
df.head()

InteriorsStyle
ListDate
Material
Agency
AreaIncomeType
EnvRating
PRIMEUNIT
Channel
PlotType
Architecture
Region
SubModel
Facade
State
RegionType


,Junk,PriceIndex8,PriceIndex9,PriceIndex7,ExpeditedListing,PriceIndex4,PriceIndex1,PriceIndex6,Zip,InsurancePremiumIndex,PriceIndex3,PriceIndex5,NormalisedPopulation,BuildYear,PropertyAge,PriceIndex2,Material_PRN0,Material_C8A4,Material_ZCI3,Material_NYFD,Agency_CAT1,Agency_CAT2,Agency_CAT3,Agency_OTHER,AreaIncomeType_A,AreaIncomeType_B,Channel_Direct,Channel_Other,Channel_Agent,PlotType_WHBI,PlotType_W62B,PlotType_32T2,PlotType_DIJX,PlotType_WLQ6,Architecture_YIK5,Architecture_TP7R,Architecture_I3Z9,Architecture_AVYP,Architecture_1M5X,Region_A,SubModel_FIVQ,SubModel_KT8F,SubModel_UA42,Facade_SILVER,Facade_WHITE,Facade_BLUE,Facade_GREY,Facade_BLACK,Facade_RED,Facade_GOLD,State_TX,State_FL,State_CA,State_NC,State_AZ,State_CO,State_SC,RegionType_A,RegionType_B,RegionType_C
0,0,14674.0,8270,13143.0,0,14224.0,9217.0,10387.0,21075,623,13108.0,9022.0,42077,2008,2,10692.0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,4172.0,3890,3461.0,0,4404.0,2958.0,3400.0,73129,1689,3695.0,2742.0,77258,2002,7,3615.0,1,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,8210.0,5900,6922.0,0,8952.0,4568.0,4862.0,29697,2351,7730.0,3768.0,94514,2004,6,5805.0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
3,0,7309.0,7460,6300.0,0,7460.0,5361.0,6305.0,28273,1933,6290.0,5370.0,82302,2002,7,6444.0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
4,0,9182.0,6400,8330.0,0,9846.0,5428.0,5718.0,29697,482,8583.0,4803.0,49176,2007,3,6286.0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0


thenw e are converting the catagorical column into numerical column using the cutoff which we have decided 

In [28]:
print(df.shape)
df.dropna(axis=0, inplace=True)
print(df.shape)

(62035, 60)
(61763, 60)


In [29]:
df.shape

(61763, 60)

In [30]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")



def grid_search_wrapper(refit_score='recall_score'):
    """
    fits a GridSearchCV classifier using refit_score for optimization
    prints classifier performance metrics
    """
    skf = StratifiedKFold(n_splits=10)
    grid_search = GridSearchCV(clf, param_grid, scoring=scorers, refit=refit_score,
                           cv=skf, return_train_score=True, n_jobs=-1, verbose=30)
    grid_search.fit(xtrain.values, ytrain.values)

    # make the predictions
    ypred = grid_search.predict(xtest.values)

    print('Best params for {}'.format(refit_score))
    print(grid_search.best_params_)
    #report(grid_search.cv_results_, n_top=3)
    # confusion matrix on the test data.
    print('\nConfusion matrix of Random Forest optimized for {} on the test data:'.format(refit_score))
    print(pd.DataFrame(confusion_matrix(ytest, ypred),
                 columns=['pred_neg', 'pred_pos'], index=['neg', 'pos']))
    return grid_search

Here we have created a wrapper 

this particular wrapper will predict  , it will train gridsearch and after training the gridsearch it will give the result in the form of cumalative metrics 

sow e have used different metric here 

we will be giving different metrics value and using those different metric value we can use any particular metric as result 

so what we ahve done we ahve given the stratified k fold as 10 
next we are applying gridsearch cv 

and this method will take any model it will apply the parameters , it take all the values which we give and for scoring it will use scorer and it will use refit score which ahs been given as recall score 

so when we fill refit the model to get the final model , the best model out of it , it will give by using the best score which we have set  recall score 

so we have done this because we are assuming that for us , any junk property that comes in our listing then we will have to face certain losses so that is why we want to make sure that we identify these properties correctly 
so we want to increase our recall 
so that is why we have given our score as recall

In [31]:
xtrain, xtest, ytrain, ytest = train_test_split(df[df.columns[1:]],df.Junk, test_size=0.3, stratify=df.Junk)


In [14]:

#deciding Beta value for model :

clf = RandomForestClassifier()
param_grid = {
    "class_weight":[{1:0.122,0:0.877}],
    "n_estimators":[500],
    #'min_samples_split': [3, 5, 10], 
    #'max_depth': [10, 20, 30],
    #'max_features': [3, 5, 10, 20]
}

for beta_val in {0.5,0.7,1,1.5,2,5,10,20,40,70,90}:
  scorers = {
      'precision_score': make_scorer(precision_score),
      'recall_score': make_scorer(recall_score),
      'accuracy_score': make_scorer(accuracy_score),
      'fbeta_score': make_scorer(fbeta_score, beta=beta_val)
  }
  print('*********************************************************************************')
  print('__________________________________',beta_val,'________________________________________')
  grid_search_clf = grid_search_wrapper(refit_score='fbeta_score')
  print('*********************************************************************************') 


*********************************************************************************
__________________________________ 0.5 ________________________________________
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:  6.3min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  7.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  7.8min finished


Best params for fbeta_score
{'class_weight': {1: 0.122, 0: 0.877}, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for fbeta_score on the test data:
     pred_neg  pred_pos
neg     16165        92
pos      1821       451
*********************************************************************************
*********************************************************************************
__________________________________ 0.7 ________________________________________
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:  6.3min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  7.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  7.8min finished


Best params for fbeta_score
{'class_weight': {1: 0.122, 0: 0.877}, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for fbeta_score on the test data:
     pred_neg  pred_pos
neg     16161        96
pos      1825       447
*********************************************************************************
*********************************************************************************
__________________________________ 2 ________________________________________
Fitting 10 folds for each of 1 candidates, totalling 10 fits


...

Best params for fbeta_score
{'class_weight': {1: 0.122, 0: 0.877}, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for fbeta_score on the test data:
     pred_neg  pred_pos
neg     16169        88
pos      1828       444
*********************************************************************************
*********************************************************************************
__________________________________ 20 ________________________________________
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:  6.5min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  8.0min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  8.0min finished


Best params for fbeta_score
{'class_weight': {1: 0.122, 0: 0.877}, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for fbeta_score on the test data:
     pred_neg  pred_pos
neg     16170        87
pos      1845       427
*********************************************************************************
*********************************************************************************
__________________________________ 90 ________________________________________
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:  6.4min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  8.0min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  8.0min finished


Best params for fbeta_score
{'class_weight': {1: 0.122, 0: 0.877}, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for fbeta_score on the test data:
     pred_neg  pred_pos
neg     16169        88
pos      1819       453
*********************************************************************************


So herew e have created certain scorer 

so here one scoree which taking precision , recall, accuracy, fbeta score 
so we are checking all of these score 

so for fbeta we are not aware what fbeta we should give so we are trying different beeta values 

so what we are doing is we are taking fbeta score and based on the f beta score we are trying to find out the perfect beeta value 

so we ahve ran all the models using the gridsearch wrapper again and again because gridsearch wrapper will allows to find out the best fbeta score because it is fitting the model again n again based on the best performing fbeta


sow e will get f beta score and confusio matrix also from there we can identify the false negative and trye positive 

In [44]:
for beta_val in {200,500,1000,1500,2000}:
  scorers = {
      'precision_score': make_scorer(precision_score),
      'recall_score': make_scorer(recall_score),
      'accuracy_score': make_scorer(accuracy_score),
      'fbeta_score': make_scorer(fbeta_score, beta=beta_val)
  }
  print('*********************************************************************************')
  print('__________________________________',beta_val,'________________________________________')
  grid_search_clf = grid_search_wrapper(refit_score='fbeta_score')
  print('*********************************************************************************')

*********************************************************************************
__________________________________ 200 ________________________________________
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:  6.3min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  7.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  7.8min finished


Best params for fbeta_score
{'class_weight': {1: 0.122, 0: 0.877}, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for fbeta_score on the test data:
     pred_neg  pred_pos
neg     16132       125
pos      1817       455
*********************************************************************************
*********************************************************************************
__________________________________ 1000 ________________________________________
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:  6.3min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  7.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  7.8min finished


Best params for fbeta_score
{'class_weight': {1: 0.122, 0: 0.877}, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for fbeta_score on the test data:
     pred_neg  pred_pos
neg     16145       112
pos      1837       435
*********************************************************************************
*********************************************************************************
__________________________________ 2000 ________________________________________
Fitting 10 folds for each of 1 candidates, totalling 10 fits


...

Best params for fbeta_score
{'class_weight': {1: 0.122, 0: 0.877}, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for fbeta_score on the test data:
     pred_neg  pred_pos
neg     16133       124
pos      1843       429
*********************************************************************************
*********************************************************************************
__________________________________ 500 ________________________________________
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:  6.3min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  7.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  7.9min finished


Best params for fbeta_score
{'class_weight': {1: 0.122, 0: 0.877}, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for fbeta_score on the test data:
     pred_neg  pred_pos
neg     16144       113
pos      1828       444
*********************************************************************************
*********************************************************************************
__________________________________ 1500 ________________________________________
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:  6.3min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  7.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  7.9min finished


Best params for fbeta_score
{'class_weight': {1: 0.122, 0: 0.877}, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for fbeta_score on the test data:
     pred_neg  pred_pos
neg     16132       125
pos      1830       442
*********************************************************************************


In [46]:
from xgboost import XGBClassifier
clf = XGBClassifier()

param_grid = {
        'learning_rate': [0.9],
        'class_weight': [{1: 0.122, 0: 0.877}], 'n_estimators': [500],
        #'max_depth': [3, 5, 7, 10,15]
        #'min_child_weight': [1, 3, 5],
        #'subsample': [0.5, 0.7],
        #'colsample_bytree': [0.5, 0.7]
}

for beta_val in {0,1,100,200,500,1000}:
  scorers = {
      'precision_score': make_scorer(precision_score),
      'recall_score': make_scorer(recall_score),
      'accuracy_score': make_scorer(accuracy_score),
      'fbeta_score': make_scorer(fbeta_score, beta=beta_val)
  }
  print('*********************************************************************************')
  print('__________________________________',beta_val,'________________________________________')
  grid_search_clf = grid_search_wrapper(refit_score='fbeta_score')
  print('*********************************************************************************')

*********************************************************************************
__________________________________ 0 ________________________________________
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   52.6s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   52.7s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:  3.4min remaining:   51.7s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  4.3min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  4.3min finished


Best params for fbeta_score
{'class_weight': {1: 0.122, 0: 0.877}, 'learning_rate': 0.9, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for fbeta_score on the test data:
     pred_neg  pred_pos
neg     15686       571
pos      1691       581
*********************************************************************************
*********************************************************************************
__________________________________ 1 ________________________________________
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   55.0s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   55.1s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:  3.5min remaining:   52.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  4.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  4.4min finished


Best params for fbeta_score
{'class_weight': {1: 0.122, 0: 0.877}, 'learning_rate': 0.9, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for fbeta_score on the test data:
     pred_neg  pred_pos
neg     15686       571
pos      1691       581
*********************************************************************************
*********************************************************************************
__________________________________ 100 ________________________________________
Fitting 10 folds for each of 1 candidates, totalling 10 fits


...

Best params for fbeta_score
{'class_weight': {1: 0.122, 0: 0.877}, 'learning_rate': 0.9, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for fbeta_score on the test data:
     pred_neg  pred_pos
neg     15686       571
pos      1691       581
*********************************************************************************
*********************************************************************************
__________________________________ 1000 ________________________________________
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   51.7s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   51.8s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:  3.4min remaining:   51.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  4.3min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  4.3min finished


Best params for fbeta_score
{'class_weight': {1: 0.122, 0: 0.877}, 'learning_rate': 0.9, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for fbeta_score on the test data:
     pred_neg  pred_pos
neg     15686       571
pos      1691       581
*********************************************************************************
*********************************************************************************
__________________________________ 500 ________________________________________
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   51.3s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   51.4s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:  3.5min remaining:   52.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  4.3min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  4.3min finished


Best params for fbeta_score
{'class_weight': {1: 0.122, 0: 0.877}, 'learning_rate': 0.9, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for fbeta_score on the test data:
     pred_neg  pred_pos
neg     15686       571
pos      1691       581
*********************************************************************************


In [32]:
scorers = {
     'precision_score': make_scorer(precision_score),
     'recall_score': make_scorer(recall_score),
     'accuracy_score': make_scorer(accuracy_score),
     'fbeta_score': make_scorer(fbeta_score, beta=200),
     'roc_auc_score':make_scorer(roc_auc_score)
}
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier



In [20]:
clf = RandomForestClassifier()
param_grid = {
    "class_weight":[{1:0.122,0:0.877}],
    "n_estimators":[500],
    #'min_samples_split': [3, 5, 10], 
    #'max_depth': [3, 5, 15, 25],
    #'max_features': [3, 5, 10, 20]
}
grid_search_clf = grid_search_wrapper(refit_score='precision_score')


Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:  6.3min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  8.0min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  8.0min finished


Best params for precision_score
{'class_weight': {1: 0.122, 0: 0.877}, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for precision_score on the test data:
     pred_neg  pred_pos
neg     16133       124
pos      1817       455


In [ ]:
clf = RandomForestClassifier()
param_grid = {
    "class_weight":[{1:0.122,0:0.877}],
    "n_estimators":[500],
    #'min_samples_split': [3, 5, 10], 
    #'max_depth': [3, 5, 15, 25],
    #'max_features': [3, 5, 10, 20]
}
grid_search_clf = grid_search_wrapper(refit_score='recall_score')

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:  6.2min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  7.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  7.8min finished


Best params for recall_score
{'class_weight': {1: 0.122, 0: 0.877}, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for recall_score on the test data:
     pred_neg  pred_pos
neg     16140       117
pos      1827       445


In [30]:
clf = RandomForestClassifier()
param_grid = {
    "class_weight":[{1:0.122,0:0.877}],
    "n_estimators":[500],
    #'min_samples_split': [3, 5, 10], 
    'max_depth': [3, 5, 15, 25],
    #'max_features': [3, 5, 10, 20]
}
grid_search_clf = grid_search_wrapper(refit_score='fbeta_score')

Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   44.3s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   44.4s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  3

Best params for fbeta_score
{'class_weight': {1: 0.122, 0: 0.877}, 'max_depth': 25, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for fbeta_score on the test data:
     pred_neg  pred_pos
neg     16183        74
pos      1887       385


In [33]:
!pip install rfpimp
from rfpimp import *


rfpimp will give the importance of our feature 

In [35]:

# adding a column in X with random values to help find out correct feature importance and features to be removed.
xtrain['random']=np.random.choice(range(xtrain.shape[0]), xtrain.shape[0])
xtest['random']=np.random.choice(range(xtrain.shape[0]), xtest.shape[0])

So let's say we have 70 column we ant to find out fetaure importantance  column form that 

now when we are fiding out feature importance we are not sure that how amny top features are actually important and how many bottom feature are actually unimportant so there is no line defined as this is good or this is bad 

so if we had something that id then surely that would be a random variable and that is why it is  unimportant feature 

so we are intorducing the random variable set as random now we will train our model by invluding this variable also and then we will find out the feature importance of our variables 

so after including random variables there are 60 features 

so teher will be some variable let's say 20 variable which will be above this random variable in feature importance 
so these will be actually important because they don't have random behaviour 

and whatever comes below the random variable are compeletely random varaible because they are having less importance then random varaible so these are not important for use we can get rid from them 

In [36]:
from xgboost import XGBClassifier
clf = XGBClassifier()
rf=XGBClassifier(base_score=0.5, booster='gbtree',
              class_weight={0: 0.877, 1: 0.122}, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.9, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [37]:
xtest.shape

(18529, 60)

In [38]:
rf.fit(xtrain, ytrain)

XGBClassifier(base_score=0.5, booster='gbtree',
              class_weight={0: 0.877, 1: 0.122}, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.9, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [39]:
list(zip(df.columns,rf.feature_importances_))
imp = importances(rf, xtrain, ytrain, n_samples=-1) # this function comes from package rfpimp
viz = plot_importances(imp)
viz.view()

In [40]:
imp

,Importance
Feature,
PriceIndex7,0.133298
PriceIndex6,0.089860
AreaIncomeType_B,0.087801
AreaIncomeType_A,0.081140
PriceIndex1,0.059837
PriceIndex8,0.050747
PriceIndex5,0.048804
PriceIndex3,0.042374
PriceIndex4,0.040177


In [41]:
indx= imp[imp.values > 0.018943].index.values
xtrain[indx]

,PriceIndex7,PriceIndex6,AreaIncomeType_B,AreaIncomeType_A,PriceIndex1,PriceIndex8,PriceIndex5,PriceIndex3,PriceIndex4,PriceIndex2,PriceIndex9,Zip,NormalisedPopulation
61239,4434.0,4910.0,1,0,3643.0,5803.0,3643.0,4434.0,5803.0,4910.0,4175,87105,84876
22896,4038.0,4105.0,0,1,3276.0,4933.0,3276.0,4038.0,4933.0,4105.0,5180,77041,58119
21770,5964.0,3984.0,1,0,2200.0,7078.0,3197.0,4798.0,6334.0,3313.0,4450,29532,108042
2859,9753.0,9325.0,1,0,8619.0,10571.0,8568.0,9809.0,10623.0,9373.0,8300,74135,45610
22635,3633.0,3778.0,0,1,3514.0,4580.0,2901.0,4295.0,5527.0,4655.0,5155,80229,81733
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30794,12546.0,9471.0,1,0,8586.0,13669.0,8411.0,11953.0,13553.0,9735.0,8435,73108,46078
52165,4615.0,4944.0,0,0,3810.0,5840.0,3810.0,4615.0,5840.0,4944.0,4415,28273,70504
49788,8597.0,7292.0,1,0,3652.0,10372.0,5525.0,7504.0,8356.0,4857.0,4100,80022,53109
11522,8879.0,7922.0,0,1,6743.0,9606.0,6167.0,9401.0,10349.0,7696.0,7730,80022,79501


In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier()

param_grid = {
        'learning_rate': [0.01, 0.05,0.1,0.25, 0.5, 0.7],
        'class_weight': [{1: 0.122, 0: 0.877}], 'n_estimators': [500]
        #'max_depth': [3, 5, 7, 10],
        #'min_child_weight': [1, 3, 5],
        #'subsample': [0.5, 0.7],
        #'colsample_bytree': [0.5, 0.7]
}

grid_search_clf = grid_search_wrapper(refit_score='recall_score')

gsf= grid_search_clf.best_estimator_
gsf.fit(xtrain[indx],ytrain)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   45.3s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   45.4s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  5

Best params for recall_score
{'class_weight': {1: 0.122, 0: 0.877}, 'learning_rate': 0.7, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for recall_score on the test data:
     pred_neg  pred_pos
neg     15808       449
pos      1705       567


XGBClassifier(base_score=0.5, booster='gbtree',
              class_weight={0: 0.877, 1: 0.122}, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.7, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier()

param_grid = {
        'learning_rate': [ 0.7,0.9,0.8],
        'class_weight': [{1: 0.122, 0: 0.877}], 'n_estimators': [500],
        'max_depth': [3, 5, 7, 10,15]
        #'min_child_weight': [1, 3, 5],
        #'subsample': [0.5, 0.7],
        #'colsample_bytree': [0.5, 0.7]
}

grid_search_clf = grid_search_wrapper(refit_score='recall_score')

gsf= grid_search_clf.best_estimator_
gsf.fit(xtrain[indx],ytrain)


Fitting 10 folds for each of 15 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   42.9s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   42.9s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  7

Best params for recall_score
{'class_weight': {1: 0.122, 0: 0.877}, 'learning_rate': 0.9, 'max_depth': 5, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for recall_score on the test data:
     pred_neg  pred_pos
neg     15631       626
pos      1673       599


XGBClassifier(base_score=0.5, booster='gbtree',
              class_weight={0: 0.877, 1: 0.122}, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.9, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier()

param_grid = {
        'learning_rate': [ 0.9],
        'class_weight': [{1: 0.122, 0: 0.877}], 'n_estimators': [500],
        'max_depth': [5],
        'min_child_weight': [1, 3, 5,10,15,50,100,75,125]
        #'subsample': [0.5, 0.7],
        #'colsample_bytree': [0.5, 0.7]
}

grid_search_clf = grid_search_wrapper(refit_score='recall_score')

gsf= grid_search_clf.best_estimator_
gsf.fit(xtrain[indx],ytrain)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed: 11

Best params for recall_score
{'class_weight': {1: 0.122, 0: 0.877}, 'learning_rate': 0.9, 'max_depth': 5, 'min_child_weight': 10, 'n_estimators': 500}

Confusion matrix of Random Forest optimized for recall_score on the test data:
     pred_neg  pred_pos
neg     15612       645
pos      1647       625


XGBClassifier(base_score=0.5, booster='gbtree',
              class_weight={0: 0.877, 1: 0.122}, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.9, max_delta_step=0, max_depth=5,
              min_child_weight=10, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier()

param_grid = {
        'learning_rate': [ 0.9],
        'class_weight': [{1: 0.122, 0: 0.877}], 'n_estimators': [500],
        'max_depth': [5],
        'min_child_weight': [10],
        'subsample': [0.2,0.3,0.5,0.6, 0.7,0.9],
        #'colsample_bytree': [0.5, 0.7]
}

grid_search_clf = grid_search_wrapper(refit_score='recall_score')

gsf= grid_search_clf.best_estimator_
gsf.fit(xtrain[indx],ytrain)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  9

Best params for recall_score
{'class_weight': {1: 0.122, 0: 0.877}, 'learning_rate': 0.9, 'max_depth': 5, 'min_child_weight': 10, 'n_estimators': 500, 'subsample': 0.3}

Confusion matrix of Random Forest optimized for recall_score on the test data:
     pred_neg  pred_pos
neg     14653      1604
pos      1641       631


XGBClassifier(base_score=0.5, booster='gbtree',
              class_weight={0: 0.877, 1: 0.122}, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.9, max_delta_step=0, max_depth=5,
              min_child_weight=10, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.3, verbosity=1)

In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier()

param_grid = {
        'learning_rate': [ 0.9],
        'class_weight': [{1: 0.122, 0: 0.877}], 'n_estimators': [500],
        'max_depth': [5],
        'min_child_weight': [10],
        'subsample': [0.3],
        'scale_pos_weight': [0.2,0.3,0.5,0.6, 0.7,0.9]
}

grid_search_clf = grid_search_wrapper(refit_score='recall_score')

gsf= grid_search_clf.best_estimator_
gsf.fit(xtrain[indx],ytrain)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed: 10

Best params for recall_score
{'class_weight': {1: 0.122, 0: 0.877}, 'learning_rate': 0.9, 'max_depth': 5, 'min_child_weight': 10, 'n_estimators': 500, 'scale_pos_weight': 0.9, 'subsample': 0.3}

Confusion matrix of Random Forest optimized for recall_score on the test data:
     pred_neg  pred_pos
neg     14716      1541
pos      1609       663


XGBClassifier(base_score=0.5, booster='gbtree',
              class_weight={0: 0.877, 1: 0.122}, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.9, max_delta_step=0, max_depth=5,
              min_child_weight=10, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=0.9, seed=None,
              silent=None, subsample=0.3, verbosity=1)

In [59]:
from xgboost import XGBClassifier
clf = XGBClassifier()

param_grid = {
        'learning_rate': [ 0.9],
        'class_weight': [{1: 0.122, 0: 0.877}], 'n_estimators': [500],
        'max_depth': [5],
        'min_child_weight': [10],
        'subsample': [0.3],
        'scale_pos_weight': [0.2,0.3,0.5,0.6, 0.7,0.9]
}

grid_search_clf = grid_search_wrapper(refit_score='recall_score')

gsf= grid_search_clf.best_estimator_
gsf.fit(xtrain[indx],ytrain)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed: 10

Best params for recall_score
{'class_weight': {1: 0.122, 0: 0.877}, 'learning_rate': 0.9, 'max_depth': 5, 'min_child_weight': 10, 'n_estimators': 500, 'scale_pos_weight': 0.9, 'subsample': 0.3}

Confusion matrix of Random Forest optimized for recall_score on the test data:
     pred_neg  pred_pos
neg     14661      1596
pos      1655       617


XGBClassifier(base_score=0.5, booster='gbtree',
              class_weight={0: 0.877, 1: 0.122}, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.9, max_delta_step=0, max_depth=5,
              min_child_weight=10, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=0.9, seed=None,
              silent=None, subsample=0.3, verbosity=1)

In [60]:
from xgboost import XGBClassifier
clf = XGBClassifier()

param_grid = {
        'learning_rate': [ 0.9],
        'class_weight': [{1: 0.122, 0: 0.877}], 'n_estimators': [500],
        'max_depth': [5],
        'min_child_weight': [10],
        'subsample': [0.3],
        'scale_pos_weight': [0.8,0.9,1,9]
}

grid_search_clf = grid_search_wrapper(refit_score='roc_auc_score')

gsf= grid_search_clf.best_estimator_



Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed: 10

Best params for roc_auc_score
{'class_weight': {1: 0.122, 0: 0.877}, 'learning_rate': 0.9, 'max_depth': 5, 'min_child_weight': 10, 'n_estimators': 500, 'scale_pos_weight': 0.9, 'subsample': 0.3}

Confusion matrix of Random Forest optimized for roc_auc_score on the test data:
     pred_neg  pred_pos
neg     14661      1596
pos      1655       617


In [42]:

gsf=XGBClassifier(base_score=0.5, booster='gbtree',
              class_weight={0: 0.877, 1: 0.122}, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.9, max_delta_step=0, max_depth=5,
              min_child_weight=10, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=0.9, seed=None,
              silent=None, subsample=0.3, verbosity=1)

In [61]:
gsf.fit(xtrain[indx],ytrain)
preds = gsf.predict(xtest[indx])
roc_auc_score(ytest, preds)

XGBClassifier(base_score=0.5, booster='gbtree',
              class_weight={0: 0.877, 1: 0.122}, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.9, max_delta_step=0, max_depth=5,
              min_child_weight=10, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=0.9, seed=None,
              silent=None, subsample=0.3, verbosity=1)

In [53]:
from xgboost import XGBClassifier
xgb_model= XGBClassifier()
param_tuning = {
        'learning_rate': [0.25],
        "class_weight":[{0:0.877,1:0.123}],
        'max_depth': [3],
        'min_child_weight': [5],
        'subsample': [0.7],
        'colsample_bytree': [0.05,0.1,0.2,0.5, 0.7,0.9],
        'n_estimators' : [500] 
}

xgbsearch3 = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           cv = 5,scoring ='roc_auc',
                           n_jobs = 20,
                           verbose = 30)

xgbsearch3.fit(xtrain[indx],ytrain)
print(xgbsearch3.best_params_)
report(xgbsearch3.cv_results_,5)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:   27.2s
[Parallel(n_jobs=20)]: Done   3 out of  30 | elapsed:   27.4s remaining:  4.1min
[Parallel(n_jobs=20)]: Done   5 out of  30 | elapsed:   28.4s remaining:  2.4min
[Parallel(n_jobs=20)]: Done   7 out of  30 | elapsed:  1.9min remaining:  6.3min
[Parallel(n_jobs=20)]: Done   9 out of  30 | elapsed:  1.9min remaining:  4.5min
[Parallel(n_jobs=20)]: Done  11 out of  30 | elapsed:  2.2min remaining:  3.8min
[Parallel(n_jobs=20)]: Done  13 out of  30 | elapsed:  2.2min remaining:  2.9min
[Parallel(n_jobs=20)]: Done  15 out of  30 | elapsed:  2.2min remaining:  2.2min
[Parallel(n_jobs=20)]: Done  17 out of  30 | elapsed:  2.8min remaining:  2.2min
[Parallel(n_jobs=20)]: Done  19 out of  30 | elapsed:  2.8min remaining:  1.6min
[Parallel(n_jobs=20)]: Done  21 out of  30 | elapsed:  3.1min remaining:  1.3min
[Parallel(n_jobs=20)]: Done  23 out of  30 | elapse

{'class_weight': {0: 0.877, 1: 0.123}, 'colsample_bytree': 0.5, 'learning_rate': 0.25, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 500, 'subsample': 0.7}
Model with rank: 1
Mean validation score: 0.727 (std: 0.00236)
Parameters: {'class_weight': {0: 0.877, 1: 0.123}, 'colsample_bytree': 0.5, 'learning_rate': 0.25, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 500, 'subsample': 0.7}

Model with rank: 2
Mean validation score: 0.726 (std: 0.00308)
Parameters: {'class_weight': {0: 0.877, 1: 0.123}, 'colsample_bytree': 0.7, 'learning_rate': 0.25, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 500, 'subsample': 0.7}

Model with rank: 3
Mean validation score: 0.725 (std: 0.00295)
Parameters: {'class_weight': {0: 0.877, 1: 0.123}, 'colsample_bytree': 0.9, 'learning_rate': 0.25, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 500, 'subsample': 0.7}

Model with rank: 4
Mean validation score: 0.724 (std: 0.00399)
Parameters: {'class_weight': {0: 0.877, 1: 0.123}

In [55]:
xgbfinal = xgbsearch3.best_estimator_
xgbfinal.fit(xtrain[indx],ytrain)
preds=xgbfinal.predict(xtest[indx])
print(confusion_matrix(ytest,preds))

[[16002   255]
 [ 1773   499]]
